# 第5章 推論

このレッスンでは、商品レビューやニュース記事から感情とトピックを推論します。


<div class="toc">
 <ul class="toc-item">
     <li><span><a href="#一序論" data-toc-modified-id="一、序論">一、序論</a></span></li>
     <li>
         <span><a href="#二感情推論と情報抽出" data-toc-modified-id="二、感情推論と情報抽出">二、感情推論と情報抽出</a></span>
         <ul class="toc-item">
             <li><span><a href="#21-感情傾向分析" data-toc-modified-id="2.1 感情傾向分析">2.1 感情傾向分析</a></span></li> 
             <li><span><a href="#22-感情タイプの識別" data-toc-modified-id="2.2 感情タイプの識別">2.2 感情タイプの識別</a></span></li>
             <li><span><a href="#23-怒りの識別" data-toc-modified-id="2.3 怒りの識別">2.3 怒りの識別</a></span></li>
             <li><span><a href="#24-商品情報抽出" data-toc-modified-id="2.4 商品情報抽出">2.4 商品情報抽出</a></span></li>
             <li><span><a href="#25-包括的なタスク完了" data-toc-modified-id="2.5 包括的なタスク完了">2.5 包括的なタスク完了</a></span></li>
             </ul>
         </li>
     <li><span><a href="#三トピック推論" data-toc-modified-id="三、トピック推論">三、トピック推論</a></span></li>
     <ul class="toc-item">
             <li><span><a href="#31-討論トピックの推論" data-toc-modified-id="3.1 討論トピックの推論">3.1 討論トピックの推論</a></span></li> 
             <li><span><a href="#32-特定トピックのニュースアラート作成" data-toc-modified-id="3.2 特定トピックのニュースアラート作成">3.2 特定トピックのニュースアラート作成</a></span></li>
             </ul>
     </ul>
</div>

## 一、序論

推論タスクは、モデルがテキストを入力として受け取り、何らかの分析を実行するプロセスと考えることができます。これには、ラベルの抽出、エンティティの抽出、テキストの感情理解などが含まれます。テキストの一部から肯定的または否定的な感情を抽出したい場合、従来の機械学習ワークフローでは、ラベル付きデータセットの収集、モデルのトレーニング、クラウドでモデルをデプロイして推論を行う方法の決定が必要でした。これはうまくいく可能性がありますが、全体のワークフローを実行するには多くの作業が必要です。また、感情分析、エンティティ抽出など、各タスクについて、個別のモデルのトレーニングとデプロイが必要でした。

LLMの非常に素晴らしい特徴は、これらのタスクの多くについて、プロンプトを書くだけで結果を生成し始めることができ、大量の作業を必要としないことです。これにより、アプリケーション開発の速度が大幅に向上します。また、多くの異なるモデルをトレーニングしてデプロイする方法を理解することなく、1つのモデルと1つのAPIを使用して多くの異なるタスクを実行することもできます。

In [1]:
import openai
# 导入第三方库

openai.api_key = "sk-..."
# 设置 API_KEY, 请替换成您自己的 API_KEY

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## 二、感情推論と情報抽出
### 2.1 感情分類

ECプラットフォームのランプに関するレビューを例に、そこから伝わる感情を二分類（ポジティブ/ネガティブ）してみます。

In [3]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

In [ ]:
# 日本語
lamp_review_ja = """
私は寝室用の素敵なランプが必要で、これには追加の収納機能があり、価格もそれほど高くありませんでした。\
すぐに受け取ることができました。配送中にランプの紐が壊れてしまいましたが、会社は喜んで新しいものを送ってくれました。\
数日後に届きました。組み立ても簡単でした。部品が足りなかったのでサポートに連絡したところ、とても迅速に不足部品を送ってくれました！\
私の見るところ、Luminaは顧客と製品を非常に大切にする素晴らしい会社です！
"""

それでは、このレビューの感情を分類するプロンプトを書いてみましょう。システムにこのレビューの感情が何かを教えてもらいたい場合は、「以下の商品レビューの感情は何ですか」というプロンプトを書き、通常の区切り文字とレビューテキストなどを追加するだけです。

実行してみましょう。結果は、この商品レビューの感情が肯定的だということで、これは非常に正しいようです。このランプは完璧ではありませんでしたが、この顧客は非常に満足しているようです。これは顧客と製品を大切にする素晴らしい会社のようで、肯定的な感情が正しい答えのようです。

In [5]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive.


In [ ]:
# 日本語
prompt = f"""
以下の三重バッククォートで区切られた商品レビューの感情は何ですか？

レビューテキスト: ```{lamp_review_ja}```
"""
response = get_completion(prompt)
print(response)

より簡潔な答えを得て、後処理を容易にしたい場合は、上記のプロンプトにもう一つの指示を追加できます：*一つの単語で答えてください：「ポジティブ」または「ネガティブ」*。これにより「ポジティブ」という単語のみが出力され、出力がより統一され、後続の処理に便利になります。

In [7]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

positive


In [ ]:
prompt = f"""
以下の三重バッククォートで区切られた商品レビューの感情は何ですか？

一つの単語で答えてください：「ポジティブ」または「ネガティブ」。

レビューテキスト: ```{lamp_review_ja}```
"""
response = get_completion(prompt)
print(response)

### 2.2 感情タイプの識別

引き続きランプのレビューを使用して、別のプロンプトを試してみましょう。今度は、モデルにレビュー作者が表現している感情を識別し、5項目以下のリストにまとめてもらいます。

In [9]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

satisfied, grateful, impressed, content, pleased


In [ ]:
# 日本語
prompt = f"""
以下のレビューの作者が表現している感情を識別してください。5項目以下で答えを形式化し、カンマで区切られた単語のリストにしてください。

レビューテキスト: ```{lamp_review_ja}```
"""
response = get_completion(prompt)
print(response)

大型言語モデルは、テキストの一部から特定の要素を抽出することが非常に得意です。上記の例では、レビューで表現された感情が顧客が特定の製品をどのように見ているかを理解するのに役立ちます。

### 2.3 怒りの識別

多くの企業にとって、ある顧客が非常に怒っているかどうかを知ることは重要です。そこで次のような分類問題が生まれます：以下のレビューの作者は怒りの感情を表現していますか？もし誰かが本当に怒っているなら、カスタマーサポートやカスタマーサクセスチームが顧客に連絡して状況を理解し、顧客の問題を解決することが価値があるかもしれません。

In [11]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

No


In [12]:
# 中文
prompt = f"""
以下评论的作者是否表达了愤怒？评论用三个反引号分隔。给出是或否的答案。

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

否


上面这个例子中，客户并没有生气。注意，如果使用常规的监督学习，如果想要建立所有这些分类器，不可能在几分钟内就做到这一点。我们鼓励大家尝试更改一些这样的 Prompt ，也许询问客户是否表达了喜悦，或者询问是否有任何遗漏的部分，并看看是否可以让 Prompt 对这个灯具评论做出不同的推论。

### 2.4 商品信息提取

接下来，让我们从客户评论中提取更丰富的信息。信息提取是自然语言处理（NLP）的一部分，与从文本中提取你想要知道的某些事物相关。因此，在这个 Prompt 中，我要求它识别以下内容：购买物品和制造物品的公司名称。

同样，如果你试图总结在线购物电子商务网站的许多评论，对于这些评论来说，弄清楚是什么物品、谁制造了该物品，弄清楚积极和消极的情感，有助于追踪特定物品或制造商收获的用户情感趋势。

在下面这个示例中，我们要求它将响应格式化为一个 JSON 对象，其中物品和品牌作为键。

In [13]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp with additional storage",
  "Brand": "Lumina"
}


In [14]:
# 中文
prompt = f"""
从评论文本中识别以下项目：
- 评论者购买的物品
- 制造该物品的公司

评论文本用三个反引号分隔。将你的响应格式化为以 “物品” 和 “品牌” 为键的 JSON 对象。
如果信息不存在，请使用 “未知” 作为值。
让你的回应尽可能简短。
  
评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

{
  "物品": "卧室灯",
  "品牌": "Lumina"
}


如上所示，它会说这个物品是一个卧室灯，品牌是 Luminar，你可以轻松地将其加载到 Python 字典中，然后对此输出进行其他处理。

### 2.5 综合完成任务

提取上述所有信息使用了 3 或 4 个 Prompt ，但实际上可以编写单个 Prompt 来同时提取所有这些信息。

In [15]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp with additional storage",
  "Brand": "Lumina"
}


In [16]:
# 中文
prompt = f"""
从评论文本中识别以下项目：
- 情绪（正面或负面）
- 审稿人是否表达了愤怒？（是或否）
- 评论者购买的物品
- 制造该物品的公司

评论用三个反引号分隔。将您的响应格式化为 JSON 对象，以 “Sentiment”、“Anger”、“Item” 和 “Brand” 作为键。
如果信息不存在，请使用 “未知” 作为值。
让你的回应尽可能简短。
将 Anger 值格式化为布尔值。

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "正面",
  "Anger": false,
  "Item": "卧室灯",
  "Brand": "Lumina"
}


这个例子中，我们告诉它将愤怒值格式化为布尔值，然后输出一个 JSON。您可以自己尝试不同的变化，或者甚至尝试完全不同的评论，看看是否仍然可以准确地提取这些内容。

## 三、主题推断

大型语言模型的另一个很酷的应用是推断主题。给定一段长文本，这段文本是关于什么的？有什么话题？以以下一段虚构的报纸报道为例。

In [17]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

In [18]:
# 中文
story_zh = """
在政府最近进行的一项调查中，要求公共部门的员工对他们所在部门的满意度进行评分。
调查结果显示，NASA 是最受欢迎的部门，满意度为 95％。

一位 NASA 员工 John Smith 对这一发现发表了评论，他表示：
“我对 NASA 排名第一并不感到惊讶。这是一个与了不起的人们和令人难以置信的机会共事的好地方。我为成为这样一个创新组织的一员感到自豪。”

NASA 的管理团队也对这一结果表示欢迎，主管 Tom Johnson 表示：
“我们很高兴听到我们的员工对 NASA 的工作感到满意。
我们拥有一支才华横溢、忠诚敬业的团队，他们为实现我们的目标不懈努力，看到他们的辛勤工作得到回报是太棒了。”

调查还显示，社会保障管理局的满意度最低，只有 45％的员工表示他们对工作满意。
政府承诺解决调查中员工提出的问题，并努力提高所有部门的工作满意度。
"""

### 3.1 推断讨论主题

上面是一篇虚构的关于政府工作人员对他们工作机构感受的报纸文章。我们可以让它确定五个正在讨论的主题，用一两个字描述每个主题，并将输出格式化为逗号分隔的列表。

In [19]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: ```{story}```
"""
response = get_completion(prompt)
print(response)

government survey, public sector employees, job satisfaction, NASA, Social Security Administration


In [20]:
response.split(sep=',')

['government survey',
 ' public sector employees',
 ' job satisfaction',
 ' NASA',
 ' Social Security Administration']

In [21]:
# 中文
prompt = f"""
确定以下给定文本中讨论的五个主题。

每个主题用1-2个单词概括。

输出时用逗号分割每个主题。

给定文本: ```{story_zh}```
"""
response = get_completion(prompt)
print(response)

调查结果, NASA, 社会保障管理局, 员工满意度, 政府承诺


### 3.2 为特定主题制作新闻提醒

假设我们有一个新闻网站或类似的东西，这是我们感兴趣的主题：NASA、地方政府、工程、员工满意度、联邦政府等。假设我们想弄清楚，针对一篇新闻文章，其中涵盖了哪些主题。可以使用这样的prompt：确定以下主题列表中的每个项目是否是以下文本中的主题。以 0 或 1 的形式给出答案列表。

In [22]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [23]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: ```{story}```
"""
response = get_completion(prompt)
print(response)

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1


In [24]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


In [25]:
# 中文
prompt = f"""
判断主题列表中的每一项是否是给定文本中的一个话题，

以列表的形式给出答案，每个主题用 0 或 1。

主题列表：美国航空航天局、当地政府、工程、员工满意度、联邦政府

给定文本: ```{story_zh}```
"""
response = get_completion(prompt)
print(response)

美国航空航天局：1
当地政府：0
工程：0
员工满意度：1
联邦政府：1


有结果可见，这个故事是与关于 NASA 、员工满意度、联邦政府有关，而与当地政府的、工程学无关。这在机器学习中有时被称为 Zero-Shot （零样本）学习算法，因为我们没有给它任何标记的训练数据。仅凭 Prompt ，它就能确定哪些主题在新闻文章中有所涵盖。

如果我们想生成一个新闻提醒，也可以使用这个处理新闻的过程。假设我非常喜欢 NASA 所做的工作，就可以构建一个这样的系统，每当 NASA 新闻出现时，输出提醒。

In [26]:
topic_dict = {i.split('：')[0]: int(i.split('：')[1]) for i in response.split(sep='\n')}
if topic_dict['美国航空航天局'] == 1:
    print("提醒: 关于美国航空航天局的新消息")

提醒: 关于美国航空航天局的新消息


这就是关于推断的全部内容了，仅用几分钟时间，我们就可以构建多个用于对文本进行推理的系统，而以前则需要熟练的机器学习开发人员数天甚至数周的时间。这非常令人兴奋，无论是对于熟练的机器学习开发人员，还是对于新手来说，都可以使用 Prompt 来非常快速地构建和开始相当复杂的自然语言处理任务。